# Project 02 - Image Processing

## Thông tin sinh viên

- Họ và tên: Lê Hồng Ngọc
- MSSV: 23127236
- Lớp: 23CLC05  

## Import các thư viện liên quan

In [81]:
from PIL import Image  # for read, write image
import numpy as np     # for matrix compute
import matplotlib.pyplot as plt  # for show image
import colorsys        # for convert RGB to HSL

## Helper functions

In [ ]:
# Any optional parameters beyond the required ones should be defined with default values
path_name = ''
path_ext = ''

def read_img(img_path):
    ''' Read image from img_path
    returns a 2D image (numpy array)
    '''
    img = Image.open(img_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    global path_name, path_ext
    path_name = img_path.split('.')[0]
    path_ext = img_path.split('.')[-1]

    img_array = np.array(img)
    img.close()
    return img_array

def show_img(img_2d):
    ''' Show image
    '''
    plt.imshow(img_2d)
    plt.axis('off')
    plt.show()

def save_img(img_2d, img_path):
    ''' Save image to img_path
    '''
    # dot_index = img_path.rfind('.')
    # if dot_index == -1:
    #     new_path = img_path + '_' + func_name
    # else:
    #     new_path = img_path[:dot_index] + '_' + func_name + img_path[dot_index:]
    img_2d = np.array(img_2d).astype(np.uint8)
    plt.imsave(img_path, img_2d, vmin = 0, vmax = 255)

# 1. Adjust brightness
def adjust_brightness(img_2d):
    ''' Adjust brightness of the image
    returns a 2D image (numpy array)
    '''
    brightness = 50  # positive: increase/negative: decrease brightness
    new_img = img_2d.astype(np.float32) + brightness
    new_img = np.clip(new_img, 0, 255).astype(np.uint8)
    return new_img

# 2. Adjust contrast
def adjust_contrast(img_2d):
    ''' Adjust contrast of the image
    returns a 2D image (numpy array)
    '''
    contrast = 1.5  # >1: increase contrast/ <1 and >0: decrease contrast
    new_img = (img_2d.astype(np.float32) - 127.5) * contrast + 127.5
    new_img = np.clip(new_img, 0, 255).astype(np.uint8)
    return new_img

# 3. Flip image
# 3.1 Flip image horizontally
def flip_image_horizontal(img_2d):
    ''' Flip the image horizontally
    returns a 2D image (numpy array)
    '''
    flipped_img = np.flip(img_2d, axis=1)
    return flipped_img

# 3.2 Flip image vertically
def flip_image_vertical(img_2d):
    ''' Flip the image vertically
    returns a 2D image (numpy array)
    '''
    flipped_img = np.flip(img_2d, axis=0)
    return flipped_img

# 4. Convert RGB to grayscale/sepia
# 4.1 Convert RGB to grayscale
def convert_rgb_to_gray(img_2d):
    ''' Convert RGB image to grayscale image
    returns a 2D image (numpy array)
    '''
    gray = np.dot(img_2d, [0.299, 0.587, 0.114])
    new_img = np.stack((gray,)*3, axis=-1).astype(np.uint8)
    return new_img
    
# 4.2 Convert RGB to sepia
def convert_rgb_to_sepia(img_2d):
    ''' Convert RGB image to sepia image
    returns a 2D image (numpy array)
    '''
    sepia_img = np.array([[.393, .769, .189],
                          [.349, .686, .168],
                          [.272, .534, .131]])
    new_img = img_2d @ sepia_img.T
    new_img = np.clip(new_img, 0, 255).astype(np.uint8)
    return new_img

# 5. Blur/Sharpen image
# 5.1 Blur image
def blur_image(img_2d):
    ''' Blur the image
    returns a 2D image (numpy array)
    '''
    kernel = np.array([
        [1,  4,  6,  4, 1],
        [4, 16, 24, 16, 4],
        [6, 24, 36, 24, 6],
        [4, 16, 24, 16, 4],
        [1,  4,  6,  4, 1]
    ], dtype=np.float32) / 256.0 

    img = np.array(img_2d, dtype=np.float32)
    height, width, _ = img.shape
    pad = 5 // 2
    blur = 10 # Number of times to apply the blur
    for _ in range (blur):
        padded = np.pad(img, ((pad, pad), (pad, pad), (0, 0)), mode='edge')
        blurred = np.zeros_like(img)
        for i in range(5):
            for j in range(5):
                weight = kernel[i, j]
                blurred += weight * padded[i:i+height, j:j+width]
        img = blurred
        
    blurred_img = Image.fromarray(np.clip(img, 0, 255).astype(np.uint8))
    return blurred_img

# 5.2 Sharpen image
def sharpen_image(img_2d):
    ''' Sharpen the image
    returns a 2D image (numpy array)
    '''
    kernel = np.array([
        [ 0, -1,  0],
        [-1,  5, -1],
        [ 0, -1,  0]
    ], dtype=np.float32) 

    img = np.array(img_2d, dtype=np.float32)
    height, width, _ = img.shape
    pad = 3 // 2
    padded = np.pad(img, ((pad, pad), (pad, pad), (0, 0)), mode='edge')
    sharpened = np.zeros_like(img)
    for i in range(3):
        for j in range(3):
            weight = kernel[i, j]
            sharpened += weight * padded[i:i+height, j:j+width]

    sharpened_img = Image.fromarray(np.clip(sharpened, 0, 255).astype(np.uint8))
    return sharpened_img

# 6. Crop center of the image
def crop_center_image(img_2d):
    ''' Crop the center of the image
    returns a 2D image (numpy array)
    '''
    height, width, _ = img_2d.shape
    start_y = height // 4
    end_y = 3 * height // 4
    start_x = width // 4
    end_x = 3 * width // 4
    cropped_img = img_2d[start_y:end_y, start_x:end_x]
    return cropped_img

# 7. Crop image with a circle/ 2 ellipses 
# 7.1 Crop image with a circle
def crop_circle_image(img_2d):
    ''' Crop the image with a circle
    returns a 2D image (numpy array)
    '''
    img = np.array(img_2d).copy()
    h, w, _ = img.shape
    center_x, center_y = w // 2, h // 2
    radius = min(w, h) // 2
    Y, X = np.ogrid[:h, :w]
    distance = (X - center_x)**2 + (Y - center_y)**2
    mask = distance <= radius**2
    circle_img = np.zeros_like(img)
    circle_img[mask] = img[mask]
    return circle_img

# 7.2 Crop image with 2 ellipses
def crop_ellipse_image(img_2d):
    '''Crop the image with 2 ellipses
    returns a 2D image (numpy array)
    '''
    img = np.array(img_2d).copy()
    h, w, _ = img.shape
    center_h, center_w = h // 2, w // 2
    theta1 = 0.3 * np.pi
    theta2 = theta1 - np.pi
    Y, X = np.ogrid[:h, :w]
    X -= center_w
    Y -= center_h
    mask1 = (np.square(2 * X / w) + np.square(2 * Y / h) + (8 * np.cos(theta1) * X * Y) / (h * w)) <= np.square(np.sin(theta1))
    mask2 = (np.square(2 * X / w) + np.square(2 * Y / h) + (8 * np.cos(theta2) * X * Y) / (h * w)) <= np.square(np.sin(theta2))
    ellipse_img = np.zeros_like(img)
    ellipse_img[mask1 | mask2] = img[mask1 | mask2]
    return ellipse_img

func_map = {
    1: adjust_brightness,
    2: adjust_contrast,
    3: [flip_image_horizontal, flip_image_vertical],
    4: [convert_rgb_to_gray, convert_rgb_to_sepia],
    5: [blur_image, sharpen_image],
    6: crop_center_image,
    7: [crop_circle_image, crop_ellipse_image]
}

def process_image(img_2d, func=[1, 2, 3,...]):
    ''' Process image with a list of functions
    func: a list of functions to apply to the image
    return list processed 2D image
    '''
    processed_images = []
    global new_path
    print("Original Image:")
    show_img(img_2d)
    
    is_save = False
    if 0 in func:
        is_save = True
    for f in func:
        if f in func_map:
            if f == 0:
                continue
            elif f == 1:
                print("Adjusting brightness...")
                f1 = func_map[f](img_2d)
                processed_images.append(f1)
                show_img(f1)
            elif f == 2:
                print("Adjusting contrast...")
                f2 = func_map[f](img_2d)
                processed_images.append(f2)
                show_img(f2)
            elif f == 3:
                print("Flipping image horizontally...")
                f3_1 = func_map[f][0](img_2d)
                processed_images.append(f3_1)
                show_img(f3_1)
                print("Flipping image vertically...")
                f3_2 = func_map[f][1](img_2d)
                processed_images.append(f3_2)
                show_img(f3_2)
            elif f == 4:
                print("Converting RGB to grayscale...")
                f4_1 = func_map[f][0](img_2d)
                processed_images.append(f4_1)
                show_img(f4_1)
                print("Converting RGB to sepia...")
                f4_2 = func_map[f][1](img_2d)
                processed_images.append(f4_2)
                show_img(f4_2)
            elif f == 5:
                print("Blurring image...")
                f5_1 = func_map[f][0](img_2d)
                processed_images.append(f5_1)
                show_img(f5_1)
                print("Sharpening image...")
                f5_2 = func_map[f][1](img_2d)
                processed_images.append(f5_2)
                show_img(f5_2)
            elif f == 6:
                print("Cropping center of the image...")
                f6 = func_map[f](img_2d)
                processed_images.append(f6)
                show_img(f6)
            elif f == 7:
                print("Cropping image with a circle...")
                f7_1 = func_map[f][0](img_2d)
                processed_images.append(f7_1)
                show_img(f7_1)
                print("Cropping image with 2 ellipses...")
                f7_2 = func_map[f][1](img_2d)
                processed_images.append(f7_2)
                show_img(f7_2)

            if is_save:
                if f in [3, 4, 5, 7]:
                    new_path = f"{path_name}_{func_map[f][0].__name__}.{path_ext}"
                    save_img(processed_images[-2], new_path)
                    print(f"Image saved with new path: {new_path}")
                    new_path = f"{path_name}_{func_map[f][1].__name__}.{path_ext}"
                    save_img(processed_images[-1], new_path)
                    print(f"Image saved with new path: {new_path}")
                else:
                    new_path = f"{path_name}_{func_map[f].__name__}.{path_ext}"
                    save_img(processed_images[-1], new_path)
                    print(f"Image saved with new path: {new_path}")

    return processed_images


In [83]:
# YOUR CODE HERE

## Input image path
# path = r"C:\Users\LENOVO\Downloads\ai-generated-9562246_1280.jpg" #1280x1280
# path = r"C:\Users\LENOVO\Downloads\wilamowice-5249675_1280.jpg" #1280x853
# path = r"C:\Users\LENOVO\Downloads\ai-generated-9562246_1280_gray.jpg"
# img = read_img(path)
# print("Original Image:")
# show_img(img)

# # 1. Adjust brightness
# brightness_img = adjust_brightness(img)
# print("Brightness Adjusted Image:")
# show_img(brightness_img)

# # 2. Adjust contrast
# contrast_img = adjust_contrast(img)
# print("Contrast Adjusted Image:")
# show_img(contrast_img)

# # 3. Flip image
# # 3.1 Flip image horizontally
# flip_horizontal = flip_image_horizontal(img)
# print("Flipped Image Horizontally:")
# show_img(flip_horizontal)
# # 3.2 Flip image vertically
# flip_vertical = flip_image_vertical(img)
# print("Flipped Image Vertically:")
# show_img(flip_vertical)

# # 4. Convert RGB to grayscale/sepia
# # 4.1 
# gray_img = convert_rgb_to_gray(img)
# print("Grayscale Image:")
# show_img(gray_img)
# # 4.2
# sepia_img = convert_rgb_to_sepia(img)
# print("Sepia Image:")
# show_img(sepia_img)

# # 5. Blur/Sharpen image
# # 5.1
# blur_img = blur_image(img)
# print("Blurred Image:")
# show_img(blur_img)
# # 5.2
# sharpen_img = sharpen_image(img)
# print("Sharpened Image:")
# show_img(sharpen_img)

# # 6. Crop center of the image
# crop_img = crop_center_image(img)
# print("Cropped Image:")
# show_img(crop_img)

# # 7. Crop image with a circle/2 ellipses
# # 7.1
# crop_circle_img = crop_circle_image(img)
# print("Circular Cropped Image:")
# show_img(crop_circle_img)
# # 7.2
# ellipse_img = crop_ellipse_image(img)
# print("Elliptical Cropped Image:")
# show_img(ellipse_img)

# processed_images = process_image(img, [1, 2, 3, 4, 5, 6, 7, 0])


## Main FUNCTION

In [ ]:
# YOUR CODE HERE
def main():
    img_path = input("Enter the image path: ")


    print("\nAvailable functions:")
    for f in func_map:
        if isinstance(func_map[f], list):
            print(f"{f}: {func_map[f][0].__name__} / {func_map[f][1].__name__}")
        else:
            print(f"{f}: {func_map[f].__name__}")
    print("\n")
    
    choice = input("Enter the function numbers to apply (e.g., 1,2,3): ")
    func_list = [int(x) for x in choice.split(',')]
    print(f"Selected functions: {func_list}\n")
    process_image(read_img(img_path), func_list)

    print("Image processing complete")

if __name__ == "__main__":
    main()